In [ ]:
import os
import sys
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold 
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV, GroupKFold
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.utils import shuffle
from sklearn.model_selection import cross_val_score


In [ ]:
df_meal = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/meal/combined_features.csv', index_col=0)
df_meal = df_meal.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

df_play = pd.read_csv('/Users/andrei-macpro/Documents/Data/tracking/features/play/combined_features.csv', index_col=0)
df_play = df_play.drop(columns=['Age', 'DAI', 'Rinab', 'IQ_T2', 'duration_meal', 'duration_play','Gender'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
df_meal['label'] = df_meal['label'].map({'no_rad': 0, 'rad': 1})
df_play['label'] = df_play['label'].map({'no_rad': 0, 'rad': 1})



In [ ]:
import pandas as pd

# Reset the index
df_meal = df_meal.reset_index()

# Create the 'group' column and group by it
df_meal['group'] = df_meal['s_id'].str.split('_').str[0]
df_grouped_meal = df_meal.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_meal.index.name = 's_id'

In [ ]:
# Reset the index
df_play = df_meal.reset_index()

# Create the 'group' column and group by it
df_play['group'] = df_play['s_id'].str.split('_').str[0]
df_grouped_play = df_play.groupby('group').mean()

# Set the index back to 's_id'
# change index name to s_id
df_grouped_play.index.name = 's_id'

In [ ]:
# Reset the index of the grouped dataframes
df_grouped_meal = df_grouped_meal.reset_index()
df_grouped_play = df_grouped_play.reset_index()

# Add a new 'group' column to each DataFrame
df_grouped_meal['group'] = 'meal'
df_grouped_play['group'] = 'play'

# Concatenate the DataFrames
final_df = pd.concat([df_grouped_meal, df_grouped_play], axis=0)

final_df

In [ ]:
final_df.set_index('s_id', inplace=True)
final_df


In [ ]:
#drop last column from final_df
final_df = final_df.drop(columns=['index'])

In [ ]:
# Map 'no_rad' to 0 and 'rad' to 1
final_df['label'] = final_df['label'].map({0: 'no_rad', 1: 'rad'})

In [ ]:
final_df

In [ ]:


# Perform a grid search for each classifier
X = final_df.drop(['label','group'], axis=1)
y = final_df['label']
groups = final_df['group']

# Create a GroupKFold object
gkf = KFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]


import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)

In [ ]:
results_df.groupby('classifier').mean()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectFromModel
X = final_df.drop(['label','group'], axis=1)
y = final_df['label']
groups = final_df['group']


# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize the model
model = LogisticRegression()

# Initialize the selector
# Fit the model
model.fit(X_train, y_train)

# Get the coefficients of the model
coefficients = model.coef_[0]

# Get the absolute values of the coefficients
abs_coefficients = abs(coefficients)

# Create a DataFrame with the feature names and coefficients
feature_importances = pd.DataFrame({'feature': X.columns, 'importance': abs_coefficients})

# Sort the DataFrame by the coefficients in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Print the features in order of their importance
print(feature_importances)

In [ ]:
# Fit the model
model.fit(X_train, y_train)

# Get the coefficients of the model
coefficients = model.coef_[0]

# Create a DataFrame with the feature names and coefficients
feature_importances = pd.DataFrame({'feature': X.columns, 'coefficient': coefficients})

# Calculate the absolute values of the coefficients for the importance
feature_importances['importance'] = abs(feature_importances['coefficient'])

# Sort the DataFrame by the importance in descending order
feature_importances = feature_importances.sort_values('importance', ascending=False)

# Print the features in order of their importance, including the sign of the coefficients
print(feature_importances)

In [ ]:
feature_importances

In [ ]:
final_df.columns

In [ ]:
import pandas as pd

# Assume that 'feature_importances' is a DataFrame where the first column is the feature names
# and the second column is the feature importances

# Define the groups of features
feature_groups = {
    "movement caregiver": ['cg_movement_mean', 'cg_movement_var', 'cg_movement_std',
       'cg_movement_min', 'cg_movement_max', 'cg_movement_skew',],
    "movement child": ['child_movement_min', 'child_movement_max',
       'child_movement_skew', 'child_movement_kurt',], 
       'area travelled':['cg_area_travelled',
       'child_area_travelled'], 
       "angle child" : ['child_angle_mean',
       'child_angle_sin', 'child_angle_cosin', 'child_angle_std',
       'child_angle_var'], 
       'angle caregiver': ['cg_angle_mean', 'cg_angle_sin',
       'cg_angle_cosin', 'cg_angle_std', 'cg_angle_var'],
       'contact':['average_contact_duration',
       'frequency_contact'],
       'distance':['average_distance_duration', 'frequency_distance'],
       'proximity':['average_proximity', 'variance_proximity', 'std_dev_proximity',
       'min_proximity', 'max_proximity'],
       'correlation':['correlation'],
       'cross correlation':['cross_correlation']
}

    # Add more groups as needed

# Calculate the total absolute coefficient for each group
group_coefficients = {}
for group, features in feature_groups.items():
    group_coefficients[group] = feature_importances[feature_importances.iloc[:, 0].isin(features)].iloc[:, 1].abs().sum()

# Convert the dictionary to a DataFrame for easier plotting
group_coefficients = pd.DataFrame(list(group_coefficients.items()), columns=["Group", "Coefficient"])

# Sort the DataFrame by the coefficients in descending order
sorted_groups = group_coefficients.sort_values(by="Coefficient", ascending=False)

# Plot the group coefficients
fig, ax = plt.subplots(figsize=(20, 10))  # Increase the figure size to make the labels more readable
ax.set_title("Group coefficients")
bars = ax.bar(sorted_groups["Group"], sorted_groups["Coefficient"], align="center")

# Move the x-axis ticks to the left
ax.set_xticks(np.arange(len(sorted_groups["Group"])) - 0.2)
ax.set_xticklabels(sorted_groups["Group"], rotation=45, fontsize=14)  # Rotate the x-axis labels to make them more readable

plt.show()

In [ ]:
# Calculate the total absolute coefficient for each group
group_coefficients = {}
for group, features in feature_groups.items():
    group_coefficients[group] = feature_importances[feature_importances.iloc[:, 0].isin(features)].iloc[:, 1].abs().sum()

# Convert the dictionary to a DataFrame for easier plotting
group_coefficients = pd.DataFrame(list(group_coefficients.items()), columns=["Group", "Coefficient"])

# Sort the DataFrame by the coefficients in descending order
sorted_groups = group_coefficients.sort_values(by="Coefficient", ascending=False)

# Plot the group coefficients
fig, ax = plt.subplots(figsize=(20, 10))  # Increase the figure size to make the labels more readable
ax.set_title("Group coefficients")
bars = ax.bar(sorted_groups["Group"], sorted_groups["Coefficient"], align="center")

# Move the x-axis ticks to the left
ax.set_xticks(np.arange(len(sorted_groups["Group"])) - 0.2)
ax.set_xticklabels(sorted_groups["Group"], rotation=45, fontsize=14)  # Rotate the x-axis labels to make them more readable

plt.show()

In [ ]:
group_coefficients

In [ ]:
feature_importances

In [ ]:
final_df.columns

In [ ]:
correlation = final_df['child_angle_horizontal_mean'].corr(final_df['child_movement_mean'])
print(f"The correlation between 'child_angle_horizontal_mean' and 'child_movement_mean' is {correlation}")

In [ ]:
correlation = final_df['child_angle_horizontal_mean'].corr(final_df['child_movement_mean'])
print(f"The correlation between 'child_angle_horizontal_mean' and 'child_movement_mean' is {correlation}")

In [ ]:
correlation = final_df['child_angle_horizontal_mean'].corr(final_df['child_movement_mean'])
print(f"The correlation between 'child_angle_horizontal_mean' and 'child_movement_mean' is {correlation}")

## let's use PCA to reduce the dimensionality of the data

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

# Assuming final_df is your DataFrame and the last column is the target

# Separate out the features and the target
X = final_df.iloc[:, :-2]
y = final_df.iloc[:, -2]

# Standardize the features to have a mean=0 and variance=1
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Apply PCA to the features
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Convert the PCA results back to a DataFrame
final_df_pca = pd.DataFrame(data=X_pca)
final_df_pca['target'] = y

In [ ]:
final_df

In [ ]:


# Perform a grid search for each classifier
X = final_df.drop(['label','group'], axis=1)
y = final_df['label']
groups = final_df['group']

# Create a GroupKFold object
gkf = KFold(n_splits=5)

# Define the classifiers and their parameters
classifiers = [
('lr', LogisticRegression(), {'lr__C': [0.01, 0.1, 1, 10, 100], 'lr__penalty': ['l1', 'l2'], 'lr__solver': ['liblinear', 'saga']}),
    ('svc_linear', SVC(kernel='linear'), {'svc_linear__C': [0.01, 0.1, 1, 10, 100]}),
    ('svc_rbf', SVC(kernel='rbf'), {'svc_rbf__C': [0.01, 0.1, 1, 10, 100], 'svc_rbf__gamma': [0.01, 0.1, 1, 10, 100]}),
    ('rf', RandomForestClassifier(), {'rf__n_estimators': [10, 50, 100, 200], 'rf__max_depth': [None, 5, 10, 15], 'rf__min_samples_split': [2, 5, 10]})
]


import pandas as pd

# Initialize a list to store the results
results = []

# Perform the grid search 10 times with different random states
for i in range(10):
    # Shuffle the data with a different random state each time
    X_shuffled, y_shuffled, groups_shuffled = shuffle(X, y, groups, random_state=i)

    # Perform a grid search for each classifier
    for name, classifier, params in classifiers:
        pipeline = Pipeline([('scaler', StandardScaler()), (name, classifier)])
        grid_search = GridSearchCV(pipeline, params, cv=gkf)
        grid_search.fit(X_shuffled, y_shuffled, groups=groups_shuffled)

        # Calculate the cross-validated F1 score, precision, and recall
        f1_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='f1_macro', groups=groups_shuffled)
        precision_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='precision_macro', groups=groups_shuffled)
        recall_scores = cross_val_score(grid_search.best_estimator_, X_shuffled, y_shuffled, cv=gkf, scoring='recall_macro', groups=groups_shuffled)

        # Store the results in a dictionary and add it to the list
        results.append({
            'random_state': i,
            'classifier': name,
            'best_params': grid_search.best_params_,
            'best_score': grid_search.best_score_,
            'f1_score': f1_scores.mean(),
            'precision': precision_scores.mean(),
            'recall': recall_scores.mean()
        })

# Convert the results to a DataFrame
results_df = pd.DataFrame(results)